In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
from scipy.io import loadmat

## 0.- Function definitions

#### Preprocessing functions

In [3]:

def background_subtraction(ST_T_ComplexTensor, WindSize):
    """
    This function performs the background subtraction of consecutive ST-T complexes.

    Parameters:
        ST_T_ComplexTensor (array): Tensor with windowed ST-T complexes. (3D array)
        WindSize (int): Size of the sliding window.

    Returns:
        ST_T_TensorBacGrElim (array): Tensor of windowed ST-T complexes after background subtraction.
    """

    ST_T_TensorBacGrElim = np.zeros_like(ST_T_ComplexTensor)

    X = ST_T_ComplexTensor.copy()

    A_pattern = X[0::2, :]
    A_pattern_dis = np.zeros_like(A_pattern)
    A_pattern_dis[0:-1, :] = A_pattern[1:, :]
    A_pattern_dis[-1, :] = A_pattern[WindSize // 2 - 1, :]
    B_pattern = X[1::2, :]
    X[0::2, :] = X[0::2, :] - B_pattern
    X[1::2, :] = X[1::2, :] - A_pattern_dis
    ST_T_TensorBacGrElim = X

    return ST_T_TensorBacGrElim



In [4]:
from scipy.interpolate import splev, splrep


def bw_elimination(orig_ecg, fiducial_points, fs):
  """
  This function produces an estimate of the baseline wander using polynomial fitting by cubic splines.

  Parameters:
      orig_ecg (array): Samples of the original ECG.
      fiducial_points (array): Fiducial points.
      fs (float): Sampling frequency.

  Returns:
      bw_estimate (array): Baseline wander estimate.
  """

  tPQ = 80e-3  # We take a representative sample of the isoelectric line 80 ms before the fiducial point.
  nPQ = int(tPQ * fs)

  numb_fid_points = len(fiducial_points)
  if fiducial_points[0].any() - nPQ <= 0:
      the_knots = np.array(np.zeros(numb_fid_points+1))
      the_knots[0] = 1
      the_knots[1:numb_fid_points] = fiducial_points[1:numb_fid_points] - nPQ
      the_knots[numb_fid_points] = len(orig_ecg)
      the_knots=the_knots-1
      the_knots=the_knots.astype(int)
      c =np.concatenate((the_knots[1],the_knots[1:numb_fid_points],the_knots[numb_fid_points-1]),axis=None)

      bw_estimate = splev(np.arange(len(orig_ecg)), splrep(the_knots,orig_ecg[c]), ext=0)


  else:
      the_knots = np.array(np.zeros(numb_fid_points + 2))
      the_knots[1:numb_fid_points] = fiducial_points - nPQ
      the_knots[0] = 1
      the_knots[numb_fid_points] = len(orig_ecg)
      the_knots=the_knots-1
      the_knots=the_knots.astype(int)
      c =np.concatenate((the_knots[1],the_knots[1:numb_fid_points],the_knots[numb_fid_points-1]),axis=None)
      bw_estimate = splev(np.arange(len(orig_ecg)), splrep(the_knots,c), ext=0)


  return bw_estimate

In [5]:
from scipy.signal import kaiserord, firwin, lfilter
import numpy as np

def filtering_ST_T_segments(ST_T_ComplexTensor, Fs):
    """
    This function filters out the ST-T complexes to eliminate noise outside the TWA band (0.3-15 Hz).

    Parameters:
        ST_T_ComplexTensor (array): Tensor with windowed ST-T complexes.
        Fs (float): Working sampling frequency.

    Returns:
        ST_T_TensorFiltered (array): Tensor of windowed ST-T complexes after filtering.
    """

    # Design the filter
    nyquist = 0.5 * Fs
    passband_low = 0.3
    passband_high = 15.0
    ripple_db = 0.01
    ripple = 80.0

    # obtain numtaps and beta using kaiserord
    numtaps, beta = kaiserord(ripple, (passband_high - passband_low) / nyquist)

    # Compute the normalized cutoff frequencies
    cutoff_low = passband_low / nyquist
    cutoff_high = passband_high / nyquist

    # Compute the filter coefficients using firwin
    b = firwin(numtaps, [cutoff_low, cutoff_high], window=('kaiser', beta), pass_zero=False, scale=False)
    delay = int((numtaps + 1) / 2)
    b_wide = b

    ST_T_TensorFiltered = np.zeros_like(ST_T_ComplexTensor)  # Initialize the filtered tensor

    # Iterate over segments, windows, and ST-T samples
    for k1 in range(ST_T_ComplexTensor.shape[2]):
        for k2 in range(ST_T_ComplexTensor.shape[0]):
            # Apply filtering to each window
            filtered_wide = lfilter(b_wide, 1, np.concatenate((ST_T_ComplexTensor[k2, :, k1], np.zeros(delay))))
            ST_T_TensorFiltered[k2, :, k1] = filtered_wide[delay:ST_T_ComplexTensor.shape[1] + delay]

    return ST_T_TensorFiltered


In [6]:
def heartbeat_windowing(ST_T_complexes, ST_T_onset, WindSize, WindStep):
    """
    This function performs windowing to track alternans in an ECG.

    Parameters:
        ST_T_complexes (array): 2D matrix of ST-T segments of the signal.
        ST_T_onset (array): Onset of the ST-T segments.
        WindSize (int): Window length in beats.
        WindStep (int): Step size of the sliding process.

    Returns:
        ST_T_segments (array): Tensor with windowed ST-T complexes.
        ST_T_onset_by_segments (array): ST-T onset of the tensor.
        ST_T_dim (tuple): Dimensions of the incoming matrix of ST-T complexes.
    """

    Overlap = WindSize - WindStep
    ST_T_dim = np.shape(ST_T_complexes)

    NumbSegments = int((ST_T_dim[0] - Overlap) / WindStep)


    # Adjust the number of segments if necessary
    if NumbSegments == np.ceil((ST_T_dim[0] - Overlap) / WindStep):
        NumbSegments = NumbSegments - 1


    ST_T_segments = np.zeros((WindSize, ST_T_dim[1], NumbSegments))  # Tensor to store windowed ST-T complexes
    ST_T_onset_by_segments = np.zeros((NumbSegments, WindSize))  # ST-T onset of the tensor


    index1 = 0
    index2 = WindSize

    # Perform windowing
    for k in range(NumbSegments):
        ST_T_segments[:,:,k] = ST_T_complexes[index1:index2,:]

        ST_T_onset_by_segments[k,:] = ST_T_onset[index1:index2]
        index1 = index1 + WindStep
        index2 = index2 + WindStep

    return ST_T_segments, ST_T_onset_by_segments, ST_T_dim

In [7]:
def np_round(a, decimals=0):
    return np.round(a * 10 ** decimals) / 10 ** decimals


def ST_T_alignment(ST_T_complexes, ST_T_onset, ecg_signal, fs):

    nbeats, nsamples = ST_T_complexes.shape
    ST_T_complexes_align = np.zeros(ST_T_complexes.shape)
    # Median template
    proto = np.median(ST_T_complexes, axis=0)
    # Possible temporal variation from the initial position of each ST_T complex
    w =(0.03 * fs)  # samples
    w = np_round(w)
    w= int(w)

    for k in range(nbeats):
        ini = int(ST_T_onset[k])
        fin = int(ini + nsamples - 1)

        ecg_segment = ecg_signal[ini - w:fin + w]
        cross_corr = np.zeros(2 * w)

        for m in range(2 * w):
            temp = ecg_segment[m:-(2 * w) + m] #[m:fin-(2*w)+m-1]
            # Ajustar la dimensión de temp
            if temp.shape[0] < proto.shape[0]:
                temp = np.append(temp, temp[-1])
            cc = np.corrcoef(proto, temp)
            cross_corr[m] = cc[1, 0]

        posMax = np.argmax(cross_corr)
        ST_T_complexes_align[k, :] = ecg_segment[posMax:posMax + nsamples]    #[posMax:fin-(2*w)+posMax-1]

    return ST_T_complexes_align

In [8]:
def ST_T_delineation(ecg_signal, fiducial_points, fs):
    """
    This function delineates the ST-T complex using intervals of 300 ms length.
    The ST-T onset is chosen to be a variable distance from the QRS point that
    depends on the previous RR interval length.

    Parameters:
        ecg_signal (array): ECG signal.
        fiducial_points (array): R wave annotations.
        fs (float): Sampling frequency.

    Returns:
        ST_T_complexes (array): Matrix of ST-T complexes.
        ST_T_onset (array): Array of annotations of ST-T onset values.
    """
    RR_interval = np.diff(fiducial_points) / fs  # RR interval in seconds
    qi = 40 + (1.3 * np.sqrt(RR_interval * 1e3))  # ST-T onset from R in milliseconds

    if qi.ndim == 1:
        qi = np.column_stack((qi[0], qi[:-1]))
    else:
        qi = np.column_stack((qi[:, 0], qi[:, :-1]))
    s = np.array(qi * 1e-3 * fs)
    fiducial_points = fiducial_points.transpose()


    ST_T_onset = fiducial_points + np.ceil(s) + 1  # ST-T onset in samples

# para que no sea una lista con matrices con el mismo numero repetido
    ST_T_onset = [matriz[0] for matriz in ST_T_onset]

    ST_T_interval = np.ceil(300e-3 * fs)  # ST-T interval length in number of samples

    ST_T_complexes = np.zeros((len(ST_T_onset)-1, int(ST_T_interval)))
    last_beat = ST_T_onset[-1]

    #La variable onset representa el inicio de cada segmento y la variable interval representa la duración de cada segmento
    number_of_ST_T = len(ST_T_onset)
    for k in range(number_of_ST_T - 1):
        onset = int(np.ceil(ST_T_onset[k]))
        interval = onset + ST_T_interval

        if interval <= len(ecg_signal):
            # Asegurarse de que el índice de inicio y fin estén dentro de los límites del arreglo
            onset_index = int(onset)
            interval_index = min(int(interval), len(ecg_signal))
            ST_T_complexes[k, :] = ecg_signal[onset_index:interval_index]

    return ST_T_complexes, ST_T_onset


#### Add alternans function


In [9]:
def include_TWA(clean_ecg, ST_T_onset, Alternan_wave, TWA_pattern, Type_of_inclusion):
    """
    The purpose of this function is the inclusion of an alternan wave into an ECG signal.

    Parameters:
        clean_ecg (numpy array): Array of integers corresponding to the ECG signal.
        ST_T_onset (numpy array): Array of annotations of ST-T onset values.
        Alternan_wave (numpy array): Waveform to perform TWA.
        TWA_pattern (numpy array): Array that describes the intervals where TWA are included.
        Type_of_inclusion (str): If this string is 'alternate', the alternan wave is included in every other beats with opposite sign.
                                 Otherwise, the alternan wave is only included every two beats.

    Returns:
        ecg_with_TWA (numpy array): ECG with TWA.
        Alt_waves (numpy array): Matrix whose rows consist of the alternan waves of ecg_with_TWA. It is subsequently used to compute the ANR.
    """

    # Get the length of the Alternan wave
    L_AltWav = len(Alternan_wave)

    # Create a copy of the clean ECG signal
    ecg_with_TWA = clean_ecg.copy()

    # Initialize the Alt_waves matrix
    Alt_waves = np.zeros((len(ST_T_onset), L_AltWav))

    if Type_of_inclusion.lower() == 'alternate':
        # Include alternan wave in every other beat with opposite sign
        for k, onset in enumerate(ST_T_onset):
            if onset + L_AltWav <= (clean_ecg.shape[1]):
                Alt_waves[k, :] = TWA_pattern[k] * Alternan_wave * (-1) ** k

                ecg_with_TWA[0, onset:onset + L_AltWav] += Alt_waves[k, :]



    else:
        # Include alternan wave every two beats
        for k, onset in enumerate(ST_T_onset[1::2]):
            if onset + L_AltWav <= (clean_ecg.shape[1]):
                Alt_waves[k, :] = TWA_pattern[2 * k] * Alternan_wave
                ecg_with_TWA[0, onset:onset + L_AltWav] += Alt_waves[k, :]
    return ecg_with_TWA, Alt_waves


In [10]:
import numpy as np

def alternant_wave_inclusion(clean_ecg, TWA_pattern, type_of_inclusion, ST_T_onset, V_Alt_wave, Fs):
    """
    This function introduces an alternant wave according to the pattern
    defined by 'TWA_pattern' in the signal 'clean_ecg'.

    Input parameters:
        - clean_ecg: annotations of the R wave.
        - TWA_pattern: TWA pattern vector.
        - type_of_inclusion: manner TWA is included; can be 'alternate' or not.
        - ST_T_onset: onset of ST-T complexes of the ECG.
        - V_Alt_wave: alternant voltage.
        - Fs: sampling frequency

    Output parameters:
        - ecg_with_TWA: ECG signal with alternans.
        - Alt_waves: matrix whose rows consist of the alternan wave ecg_with_TWA.
        - Alternan_wave: waveform to perform TWA.
        - Onset_Valt: onset of TWA inclusion, having considered Jitter effect.

        _____________________________________
       |                                     |
       |  Manuel Blanco Velasco: 2023/03/06  |
       |_____________________________________|

    """

    # Load the MATLAB file
    mat_data = scipy.io.loadmat('AltWav_128hz.mat')

    AltWav = np.random.randint(1, 16)

    # Access the desired key
    Alternan_wave = mat_data['AltWav_128hz'][:, AltWav - 1]

    if type_of_inclusion.lower() == 'alternate':
        alpha = V_Alt_wave / (np.max(np.abs(2 * Alternan_wave)) * 1e3)
    else:
        alpha = V_Alt_wave / (np.max(np.abs(Alternan_wave)) * 1e3)

    jitter_std = int(0.02 * Fs)  # Jitter effect standard deviation of 20 ms (0.02*Fs samples)

    Onset_Valt = (ST_T_onset + np.round(np.random.randn(len(ST_T_onset)) * jitter_std)).astype(int)

    #Onset_Valt = ST_T_onset + np.round(np.random.normal(0, jitter_std, size=len(ST_T_onset))).astype(int)

    ecg_with_TWA, Alt_waves = include_TWA(clean_ecg, Onset_Valt, alpha * Alternan_wave, TWA_pattern, type_of_inclusion)

    V0 = np.max(np.abs(alpha * Alternan_wave)) * 1e3  # Alternan wave amplitude in microvolts
    if type_of_inclusion.lower() == 'alternate':
        V0 = 2 * V0

    return ecg_with_TWA, Alt_waves, Alternan_wave, alpha, Onset_Valt

## Analysis System

### Create a list from all of the signals

In [11]:
import os
from google.colab import drive

In [12]:
import os
from scipy.io import loadmat
import random

from google.colab import drive
drive.mount('/content/drive')
# Ruta de la carpeta en Drive
carpeta_drive = '/content/drive/MyDrive/Señales_sin_alternancias'

# Obtener la lista de archivos en la carpeta
archivos = os.listdir(carpeta_drive)
# los mezclamos
archivos = random.sample(archivos, len(archivos))
print(archivos)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Señales_sin_alternancias_nsrdb_16265_l2_s21.mat', 'Señales_sin_alternancias_nsrdb_16483_l2_s1.mat', 'Señales_sin_alternancias_nsrdb_16786_l1_s22.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s38.mat', 'Señales_sin_alternancias_nsrdb_16539_l1_s23.mat', 'Señales_sin_alternancias_nsrdb_16483_l2_s32.mat', 'Señales_sin_alternancias_nsrdb_16420_l1_s38.mat', 'Señales_sin_alternancias_nsrdb_16420_l2_s7.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s5.mat', 'Señales_sin_alternancias_nsrdb_16265_l1_s23.mat', 'Señales_sin_alternancias_nsrdb_16483_l2_s22.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s33.mat', 'Señales_sin_alternancias_nsrdb_16539_l2_s29.mat', 'Señales_sin_alternancias_nsrdb_16265_l1_s32.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s28.mat', 'Señales_sin_alternancias_nsrdb_16420_l1_s14.mat', 'Señales_sin_alternancias_nsrdb_16265_l2

In [13]:
len(archivos)

206

In [14]:
# Create a dictionary to store all data
data_dict = {}

# Iterate over each file
for archivo in archivos:
    # Build the complete file path
    ruta_archivo = os.path.join(carpeta_drive, archivo)
    print(ruta_archivo)
    # Verify whether it is a file
    if os.path.isfile(ruta_archivo):
        if archivo.endswith('.mat'):
            # Load file .mat
            signal = loadmat(ruta_archivo)

            # Access data contained in the file

            # Perfom pre-processing
            Fs = signal['Fs']
            ecg = signal['ecg']
            qrs = signal['qrs']
            QRS_ann = signal['qrs_ann']

            # Obtain ST_T_onset to add alternans
            # 1. Base line wandering elimination
            bw_estimate_clean = bw_elimination(ecg[0, :], qrs[0, :], Fs)
            clean_ecg_noBW = ecg[0, :] - bw_estimate_clean + np.mean(bw_estimate_clean)

            # 2. Segmentation of ST-T complexes (this is where alternation will be searched for)
            ST_T_complexes, ST_T_onset = ST_T_delineation(clean_ecg_noBW, qrs, Fs)

            # Associate the data to the corresponding file in the dictionary
            data_dict[archivo] = {
                'ecg': ecg,
                'ST_T_onset': ST_T_onset,
                'qrs': qrs,
                'qrs_ann': QRS_ann
            }


/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16265_l2_s21.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16483_l2_s1.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16786_l1_s22.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16483_l1_s38.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16539_l1_s23.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16483_l2_s32.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16420_l1_s38.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16420_l2_s7.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16483_l1_s5.mat
/content/drive/MyDrive/Señales_sin_alternancias/Señales_sin_alternancias_nsrdb_16265_l1_s23.mat
/content/drive/MyDrive/Señales_si

### añadir alternancia a la mitad de las señales

In [15]:
import os
from scipy.io import loadmat
import scipy

signals_with_alternans = {}

# Calculate the number of signals to signal for (half of the data set)
half_dataset_size = len(data_dict) // 2

# Get the keys corresponding to half of the dictionary
half_keys = list(data_dict.keys())[:half_dataset_size]

# Input parameters
type_of_inclusion = 'alternate
V_Alt_wave = 35 # muV -----> Specify the desired alternans voltage
Fs = 128 # Hz

for key in half_keys:
    signal = data_dict[key]['ecg']

    # Obtain the corresponding 'TWA_pattern' and 'ST_T_onset'.
    TWA_pattern = np.ones(qrs.shape[1])
    TWA_pattern = np.ones(data_dict[key]['qrs'].shape[1])
    ST_T_onset = data_dict[key]['ST_T_onset']
    QRS_ann = data_dict[key]['qrs_ann']
    qrs = data_dict[key]['qrs']
    # Generate the new signal with alternans using the function 'alternant_wave_inclusion'.
    ecg_with_TWA, alt_waves, Alternan_wave, alpha, Onset_Valt = alternant_wave_inclusion(signal, TWA_pattern, type_of_inclusion, ST_T_onset, V_Alt_wave, Fs)
    # Convert the list to a NumPy array
    st_t_onset_array = np.array(ST_T_onset)
    # Add the new signal to the signal dictionary with toggles
    signals_with_alternans [key] = {
                'ecg': ecg_with_TWA,
                'ST_T_onset': st_t_onset_array,
                'qrs': qrs,
                'qrs_ann': QRS_ann,
                'label': 1
            }

### Create separate dictionary for signals without alternans

In [16]:
#create dictionary for signals without alternans
half_dataset_size = len(data_dict) // 2
print(half_dataset_size)
half_keys = list(data_dict.keys())[half_dataset_size:]
signals_no_TWA = {key: data_dict[key] for key in half_keys}
print(signals_no_TWA.keys())
print(signals_with_alternans.keys())

103
dict_keys(['Señales_sin_alternancias_nsrdb_16483_l2_s17.mat', 'Señales_sin_alternancias_nsrdb_16420_l1_s15.mat', 'Señales_sin_alternancias_nsrdb_16420_l1_s19.mat', 'Señales_sin_alternancias_nsrdb_16273_l2_s36.mat', 'Señales_sin_alternancias_nsrdb_16420_l1_s26.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s30.mat', 'Señales_sin_alternancias_nsrdb_16539_l1_s6.mat', 'Señales_sin_alternancias_nsrdb_16539_l1_s1.mat', 'Señales_sin_alternancias_nsrdb_16539_l2_s28.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s8.mat', 'Señales_sin_alternancias_nsrdb_16265_l1_s35.mat', 'Señales_sin_alternancias_nsrdb_16265_l1_s6.mat', 'Señales_sin_alternancias_nsrdb_16420_l1_s31.mat', 'Señales_sin_alternancias_nsrdb_16420_l2_s10.mat', 'Señales_sin_alternancias_nsrdb_16273_l2_s15.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s45.mat', 'Señales_sin_alternancias_nsrdb_16483_l2_s8.mat', 'Señales_sin_alternancias_nsrdb_16265_l1_s20.mat', 'Señales_sin_alternancias_nsrdb_16483_l1_s20.mat', 'S

###label

In [ ]:
#label
for key in signals_no_TWA:
    signals_no_TWA[key]['label'] = 0


### combine signals with + without

In [18]:
# Combine signals with alternans and without alternans
merged_dict = {**signals_with_alternans, **signals_no_TWA}
print(len(merged_dict.keys()))

206


In [19]:
#signals_with_alternans

###  Do preprocessing



In [20]:
all_filtered_FINAL = {}

for key in merged_dict:
  signal = merged_dict[key]['ecg']
  qrs = merged_dict[key]['qrs']
  print(key)

# Perform preprocessing
  ecg = signal
  # 1. Base line wandering elimination
  bw_estimate_clean = bw_elimination(ecg[0,::], qrs[0,::], Fs)
  clean_ecg_noBW = ecg[0,:] - bw_estimate_clean + np.mean(bw_estimate_clean)

  # 2. Segmentation of the ST-T complexes (this is where the alternation will be searched)
  ST_T_complexes, ST_T_onset = ST_T_delineation(clean_ecg_noBW, qrs, Fs)


  # 3. Windowing (to indicate in which size of windows, with how many beats you want to do the analysis)

  WindSize = 32 # analyze in blocks of 32 heartbeats
  WindStep = 8 # each window contains 8 new beats

  ST_T_segments, ST_T_onset_by_segments, _ = heartbeat_windowing(ST_T_complexes, ST_T_onset, WindSize, WindStep)


  # Alignment of the ST-T segments

  WindSize = 32
  NumbSegments = ST_T_segments.shape[2]

  # Step 1: Create an array of zeros to store the aligned ST-T segments
  ST_T_segments_aligned = np.zeros((WindSize, ST_T_segments.shape[1], NumbSegments))
  # Step 2: Iterate over each segment
  for k in range(NumbSegments):
      # Step 3: Align the ST-T segment using ST_T_alignment function.
      aligned_segment = ST_T_alignment(ST_T_segments[:,::,k], ST_T_onset_by_segments[k,:], clean_ecg_noBW, Fs)

      # Step 4: Store the aligned segment in the corresponding position of ST_T_segments_aligned
      ST_T_segments_aligned[:,::,k] = aligned_segment


  # 5. Background Subtraction (subtract the beats 2 by 2 to keep only the alternating part if there is one)

  WindSize = 32
  NumbSegments = ST_T_segments_aligned.shape[2]

  ST_T_dif = np.zeros_like(ST_T_segments_aligned)
  for k in range(NumbSegments-1):
      ST_T_TensorBacGrElim = background_subtraction(ST_T_segments_aligned[:,:,k], WindSize)

      ST_T_dif[:,:,k] = ST_T_TensorBacGrElim


  # 6. Low pass filtering, since the information is below 15 Hz.
  fs = 128
  ST_T_dif_filtered = filtering_ST_T_segments(ST_T_dif, fs)
  segmentos = {}
  for k in range(NumbSegments):
    # Put together all the operations you want to perform with the segments

    segmentos[k] = ST_T_dif_filtered[:,:,k]

    all_filtered_FINAL[key] = segmentos


Señales_sin_alternancias_nsrdb_16265_l2_s21.mat
Señales_sin_alternancias_nsrdb_16483_l2_s1.mat
Señales_sin_alternancias_nsrdb_16786_l1_s22.mat
Señales_sin_alternancias_nsrdb_16483_l1_s38.mat
Señales_sin_alternancias_nsrdb_16539_l1_s23.mat
Señales_sin_alternancias_nsrdb_16483_l2_s32.mat
Señales_sin_alternancias_nsrdb_16420_l1_s38.mat
Señales_sin_alternancias_nsrdb_16420_l2_s7.mat
Señales_sin_alternancias_nsrdb_16483_l1_s5.mat
Señales_sin_alternancias_nsrdb_16265_l1_s23.mat
Señales_sin_alternancias_nsrdb_16483_l2_s22.mat
Señales_sin_alternancias_nsrdb_16483_l1_s33.mat
Señales_sin_alternancias_nsrdb_16539_l2_s29.mat
Señales_sin_alternancias_nsrdb_16265_l1_s32.mat
Señales_sin_alternancias_nsrdb_16483_l1_s28.mat
Señales_sin_alternancias_nsrdb_16420_l1_s14.mat
Señales_sin_alternancias_nsrdb_16265_l2_s16.mat
Señales_sin_alternancias_nsrdb_16483_l2_s3.mat
Señales_sin_alternancias_nsrdb_16539_l1_s19.mat
Señales_sin_alternancias_nsrdb_16539_l1_s2.mat
Señales_sin_alternancias

In [21]:
print(len(all_filtered_FINAL))

206


In [24]:
ruta_archivo = "/content/drive/MyDrive/FINAL_35.json"

In [25]:
import json
# Function to convert NumPy arrays to recursive lists
def numpy_to_list(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: numpy_to_list(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [numpy_to_list(v) for v in obj]
    else:
        return obj
with open(ruta_archivo, "w") as archivo:
    json.dump(numpy_to_list(all_filtered_FINAL), archivo)


with open(ruta_archivo, "r") as archivo:
    datos_cargados = json.load(archivo)